# how to run

This folder contains the dataset which will cause imaginary phonon frequencies.
The program can't fix it and stops calculation.

## step1
At first, please run 
```
$ python split_dfset.py
```
Folders 0001/ to 00014/ will be made.
Each folder contains dfset file.

## step2 
Please find the cell containing previous_it_str and it_str and 
change the lines to 
```
previous_it_str = '0000'
it_str = '0001'
```
and "Run"->"Run All Cells". 
Files will be made in 0001/. 

0001/free_energy_history.txt is the important history file.

```
'unstable': False,
'converged': False,
```
in phase1_result["results"].get_dict() will be shown.
The free energy isn't converged. 

You can see that "imaginary_ratios", which is the ratio of imaginary frequencies of all the frequencies, isn't 0 now. The program trys to make it zero with increasing the dataset size. 

# step 3
Please continue running all with changing
```
previous_it_str = '0001'
it_str = '0002'
```
and so on.

```
previous_it_str = '0008'
it_str = '0009'
```
will cause 
```
 'unstable': True,
 'converged': False,
```
It shows that the program can't fix the emergence of the imaginary frequencies.


In [109]:
import argparse
import os
import shutil
from pathlib import Path

import numpy as np
from ase.units import Ry

from aiida_alamode.workflows import PhononCalculator
from aiida_alamode.workflows.model_selection import ForceConstantModelSelector
from aiida_alamode.io import DfHistoryData
from io import TextIOWrapper


In [110]:
import aiida
from aiida.plugins import DataFactory, WorkflowFactory
from aiida.plugins.factories import CalculationFactory

from aiida.orm import Str, Int, Dict, List, Float, load_node
from aiida.engine import calcfunction, workfunction, submit, run, run_get_node
from aiida.engine import WorkChain, ToContext, if_, append_
from aiida_alamode.tools.aiida_support import get_or_create_local_computer, get_or_create_code
from os.path import expanduser
import os

from aiida.orm import Code

# must execute it in the first place
aiida.load_profile()


In [111]:
# make computer local
home = expanduser("~")
work_directory = os.path.join(home, 'aiida')
computer_local = get_or_create_local_computer(work_directory, 'localhost')

In [112]:
SinglefileData = DataFactory('singlefile')
StructureData = DataFactory('structure')
ArrayData = DataFactory('array')
FrameData = DataFactory('dataframe.frame')


In [113]:
code_anphon_calculator = get_or_create_code('alamode.anphon_calculator',
                          computer_local,
                          '/home/max/Documents/alamode/bin/anphon')

# code_anphon_calculator = CalculationFactory('alamode.anphon_calculator') # doesn't work.

type(code_anphon_calculator)

aiida.orm.nodes.data.code.Code

In [114]:
basedir = os.getcwd()
print("basedir", basedir)

poscar_file =  os.path.join(basedir, "SPOSCAR")
prim_file = os.path.join(basedir, "relaxed_final_symm.POSCAR.vasp")
previous_it_str = '0008'
it_str = '0009'
cwd = os.path.join(basedir, it_str)

dfset_file = os.path.join(cwd, "dfset")
history_file = os.path.join(basedir, previous_it_str, "free_energy_history.txt")
cwd

basedir /home/max/Documents/aiida-alamode_public/example_dxmag/free_energy_convergence/unstable,run


'/home/max/Documents/aiida-alamode_public/example_dxmag/free_energy_convergence/unstable,run/0009'

In [115]:
from ase import io
superatoms = io.read(poscar_file, format="vasp")
superstructure_node = StructureData(ase=superatoms).store()
primatoms = io.read(prim_file, format="vasp")
primstructure_node = StructureData(ase=primatoms).store()


In [116]:
dfset = np.loadtxt(dfset_file)
numatoms = len(superatoms.numbers)
dfset = dfset.reshape(-1, numatoms, 6)
dfset_node = ArrayData()
dfset_node.set_array('dfset',dfset)
dfset_node.store()


<ArrayData: uuid: 15040b18-00ab-43e3-bf8c-fbd94f3a88ef (pk: 5269)>

In [117]:
dfset.shape

(18, 96, 6)

In [118]:
history = DfHistoryData().from_file(history_file)
history_node = FrameData(df=history.df)
history_node.df

,Iteration,No. training data,best cuoff [bohr],lowest omega,ratio imaginary,Fvib (mev/atom) 0,Fvib (mev/atom) 1,Fvib (mev/atom) 2,Fvib (mev/atom) 3,Fvib (mev/atom) 4,...,Fvib (mev/atom) 11,Fvib (mev/atom) 12,Fvib (mev/atom) 13,Fvib (mev/atom) 14,Fvib (mev/atom) 15,Fvib (mev/atom) 16,Fvib (mev/atom) 17,Fvib (mev/atom) 18,Fvib (mev/atom) 19,Fvib (mev/atom) 20
0,1.0,2.0,8.48845,-45.0125,0.022719,3.735199,3.186239,0.838617,-2.923553,-7.692949,...,-56.794088,-65.267842,-74.004100,-82.982752,-92.186564,-101.600555,-111.211659,-121.008283,-130.980150,-141.118064
1,2.0,4.0,9.48845,-48.4897,0.017928,3.731067,3.154639,0.759543,-3.059012,-7.890389,...,-57.506162,-66.060742,-74.879102,-83.941060,-93.229284,-102.728736,-112.426251,-122.310220,-132.370283,-142.597158
2,3.0,6.0,12.48845,-63.1172,0.027822,3.725193,3.191823,0.878700,-2.841217,-7.563123,...,-56.250158,-64.657159,-73.325289,-82.234552,-91.367798,-100.710133,-110.248532,-119.971500,-129.868806,-139.931279
3,4.0,8.0,12.98845,-62.3009,0.027378,3.729040,3.195257,0.881857,-2.838780,-7.561792,...,-56.263325,-64.673046,-73.344011,-82.256222,-91.392501,-100.737982,-110.279627,-120.005897,-129.906590,-139.972535
4,5.0,10.0,12.98845,-62.3840,0.027329,3.727258,3.191913,0.876001,-2.847431,-7.573351,...,-56.296404,-64.709271,-73.383411,-82.298810,-91.438307,-100.786991,-110.331853,-120.061355,-129.965293,-140.034484
5,6.0,12.0,13.48845,-63.2399,0.026884,3.726473,3.188126,0.866709,-2.863063,-7.595827,...,-56.374126,-64.795568,-73.478395,-82.402596,-91.550979,-100.908663,-110.462595,-120.201281,-130.114459,-140.192976
6,7.0,14.0,15.98845,-63.0251,0.027625,3.724782,3.190807,0.876174,-2.845709,-7.569816,...,-56.275598,-64.685575,-73.356738,-82.269076,-91.405441,-100.750950,-110.292566,-120.018780,-129.919359,-139.985149
7,8.0,16.0,15.98845,-62.7899,0.028020,3.726079,3.194432,0.884340,-2.832168,-7.550418,...,-56.208434,-64.610984,-73.274608,-82.179321,-91.307962,-100.645648,-110.179370,-119.897619,-129.790162,-139.847845


# run a step

In [119]:
cwd

'/home/max/Documents/aiida-alamode_public/example_dxmag/free_energy_convergence/unstable,run/0009'

In [120]:
checkConvergenceFreeenergy_workchain = WorkflowFactory('alamode.check_convergence_freeenergy')

def phaseall(superstructure_node, primstructure_node, dfset_node, history_node, anphon_code, cwd=None,
            qspacing=0.1, conv_thr=0.1, temp=1000, cv=2, ninc=2, verbosity=1):   
    # params
    params = {"qspacing": qspacing, "conv_thr": conv_thr, "temp": temp, "cv": cv,
              "ninc": ninc, "verbosity": verbosity}
   
    inputs = {'super_structure': superstructure_node,   
              'primitive_structure': primstructure_node,
              'dfset': dfset_node,
              'params': Dict(dict=params),
              'history': history_node,
             'anphon_code': anphon_code}
    if cwd is not None:
        inputs.update({'cwd': Str(cwd)})

    result = run(checkConvergenceFreeenergy_workchain, **inputs)
    
    return result

phase1_result = phaseall(superstructure_node, primstructure_node, dfset_node, history_node, code_anphon_calculator, cwd)


Iter #9: unstable mode exists with the same cutoff radius 3-times in a row, w = -62.31 cm^-1
Stop the iteration because the system is unlikely to become stable.


In [121]:
history_file = os.path.join(cwd,  "free_energy_history.txt")

print(history_file)
df = phase1_result['history'].df
df.to_csv(history_file, index=False)

/home/max/Documents/aiida-alamode_public/example_dxmag/free_energy_convergence/unstable,run/0009/free_energy_history.txt


In [122]:
phase1_result

{'rc_best': <Float: uuid: 395abd35-b3a4-4bc5-98bf-9ca21b3bd7a4 (pk: 5274) value: 15.98845046934>,
 'fc': <ArrayData: uuid: 5100a2ce-12aa-46f7-8ce2-b9a407fb025b (pk: 5275)>,
 'ols_history': <PandasFrameData: uuid: 1e7554de-3a88-4419-9f8b-4add64a86d77 (pk: 5276)>,
 'band_results': <Dict: uuid: 93f535e5-20b3-4156-976b-5eb46a0d6a3e (pk: 5287)>,
 'bands': <SinglefileData: uuid: 179b246b-b076-4ce9-a452-0af0f6750cb3 (pk: 5286)>,
 'thermo_results': <Dict: uuid: 8ec2acf8-9fbc-4c76-8ac3-96ad5231ac29 (pk: 5297)>,
 'dosthermo_properties': <ArrayData: uuid: cb7b54a5-62eb-4dd6-a01a-32b6310a7118 (pk: 5296)>,
 'results': <Dict: uuid: 97d747b1-a774-4b7a-b828-1f499543a368 (pk: 5298)>,
 'history': <PandasFrameData: uuid: 014067a4-6cd1-4aa3-ae4a-5d860a685997 (pk: 5299)>}

In [123]:
phase1_result["history"].df

,Iteration,No. training data,best cuoff [bohr],lowest omega,ratio imaginary,Fvib (mev/atom) 0,Fvib (mev/atom) 1,Fvib (mev/atom) 2,Fvib (mev/atom) 3,Fvib (mev/atom) 4,...,Fvib (mev/atom) 11,Fvib (mev/atom) 12,Fvib (mev/atom) 13,Fvib (mev/atom) 14,Fvib (mev/atom) 15,Fvib (mev/atom) 16,Fvib (mev/atom) 17,Fvib (mev/atom) 18,Fvib (mev/atom) 19,Fvib (mev/atom) 20
0,1.0,2.0,8.48845,-45.0125,0.022719,3.735199,3.186239,0.838617,-2.923553,-7.692949,...,-56.794088,-65.267842,-74.004100,-82.982752,-92.186564,-101.600555,-111.211659,-121.008283,-130.980150,-141.118064
1,2.0,4.0,9.48845,-48.4897,0.017928,3.731067,3.154639,0.759543,-3.059012,-7.890389,...,-57.506162,-66.060742,-74.879102,-83.941060,-93.229284,-102.728736,-112.426251,-122.310220,-132.370283,-142.597158
2,3.0,6.0,12.48845,-63.1172,0.027822,3.725193,3.191823,0.878700,-2.841217,-7.563123,...,-56.250158,-64.657159,-73.325289,-82.234552,-91.367798,-100.710133,-110.248532,-119.971500,-129.868806,-139.931279
3,4.0,8.0,12.98845,-62.3009,0.027378,3.729040,3.195257,0.881857,-2.838780,-7.561792,...,-56.263325,-64.673046,-73.344011,-82.256222,-91.392501,-100.737982,-110.279627,-120.005897,-129.906590,-139.972535
4,5.0,10.0,12.98845,-62.3840,0.027329,3.727258,3.191913,0.876001,-2.847431,-7.573351,...,-56.296404,-64.709271,-73.383411,-82.298810,-91.438307,-100.786991,-110.331853,-120.061355,-129.965293,-140.034484
5,6.0,12.0,13.48845,-63.2399,0.026884,3.726473,3.188126,0.866709,-2.863063,-7.595827,...,-56.374126,-64.795568,-73.478395,-82.402596,-91.550979,-100.908663,-110.462595,-120.201281,-130.114459,-140.192976
6,7.0,14.0,15.98845,-63.0251,0.027625,3.724782,3.190807,0.876174,-2.845709,-7.569816,...,-56.275598,-64.685575,-73.356738,-82.269076,-91.405441,-100.750950,-110.292566,-120.018780,-129.919359,-139.985149
7,8.0,16.0,15.98845,-62.7899,0.028020,3.726079,3.194432,0.884340,-2.832168,-7.550418,...,-56.208434,-64.610984,-73.274608,-82.179321,-91.307962,-100.645648,-110.179370,-119.897619,-129.790162,-139.847845
8,9.0,18.0,15.98845,-62.3100,0.027427,3.727135,3.194109,0.880759,-2.839956,-7.563048,...,-56.264600,-64.674265,-73.345159,-82.257285,-91.393465,-100.738847,-110.280364,-120.006521,-129.907086,-139.972890


In [124]:
phase1_result["results"].get_dict()

{'cutoffs': [8.4884504693402,
  9.4884504693402,
  12.48845046934,
  12.98845046934,
  12.98845046934,
  13.48845046934,
  15.98845046934,
  15.98845046934],
 'unstable': True,
 'converged': False,
 'ninc_used': 2,
 'iter_start': 8,
 'exist_history': True,
 'lowest_omegas': [-45.0125,
  -48.4897,
  -63.1172,
  -62.3009,
  -62.384,
  -63.2399,
  -63.0251,
  -62.7899],
 'previous_fvib': None,
 'imaginary_ratios': [0.022718888888889,
  0.017928194444444,
  0.027822472222222,
  0.027377972222222,
  0.027328583333333,
  0.026884083333333,
  0.027624916666667,
  0.028020027777778]}